In [1]:
#import libraries

In [8]:
#Use API to pull in news data using key words 
#bloomberg, newsAPI, bing


In [ ]:
def get_headlines(keyword):
    all_headlines = []
    all_dates = []    
    date = datetime.strptime(current_date[:10], "%Y-%m-%d")
    end_date = datetime.strptime(past_date[:10], "%Y-%m-%d")
    print(f"Fetching news about '{keyword}'")
    print("*" * 30)
    while date > end_date:
        print(f"retrieving news from: {date}")
        articles = newsapi.get_everything(
            q=keyword,
            from_param=str(date),
            to=str(date),
            language="en",
            sort_by="relevancy",
            page=1,
        )
        headlines = []
        for i in range(0, len(articles["articles"])):
            headlines.append(articles["articles"][i]["title"])
        all_headlines.append(headlines)
        all_dates.append(date)
        date = date - timedelta(days=1)
    return all_headlines, all_dates

In [3]:
#put it in a dataframe

In [4]:
#segment/cut dataframe by date of article.  Drop unnecessary data columns 

In [5]:
#combine or classify articles by date

In [6]:
#Data cleaning to news content as needed: lowercase, lemmantize

In [ ]:
#TD-IDF word counts by date 